# CNNの可視化 (Gradient-weighted Class Activation Mapping; Grad-CAM)

---

## 目的
Gradient-weighted Class Activation Mapping (Grad-CAM)の仕組みを理解する.

Grad-CAMを用いてCIFAR-10データセットに対するネットワークの判断根拠の可視化を行う．

## Gradient-weighted Class Activation Mapping (Grad-CAM)
Grad-CAM[1]は，逆伝播時の正値の勾配を用いることでCNNを可視化する手法です．
Grad-CAMは，逆伝播時の特定のクラスにおける勾配をGlobal Average Pooling (GAP)[2]により空間方向に対する平均値を求め，各特徴マップに対する重みとします．
その後，獲得した重みを各特徴マップに重み付けすることでAttention map を獲得します．
02_CAM.ipynbで使用したClass Activation Mapping (CAM)[3]は，ネットワークの一部をGAPに置き換える必要があるため，Attention mapを獲得するためにネットワークを学習させる必要があります．一方で，Grad-CAMはネットワークの順伝播時の特徴マップと逆伝播時の勾配を用いてAttention mapを獲得します．そのため，学習済みの様々なネットワークからAttention map を獲得することができます．


<img src="https://www.dropbox.com/s/x23sm70ftoo7caa/grad-cam.png?dl=1" width = 100%>


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
今回は，機械学習ライブラリであるPytorchを使用します．
PyTorchとは，Python向けのオープンソース機械学習ライブラリで，Facebookに開発されました．

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchsummary
from torchvision import datasets, models

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．
下記のコードを実行してGPU情報を確認します． GPUの確認を行うためには，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPytorchで行うことが可能です． CPUとなっている場合は，上記に記載している手順にしたがって，設定を変更してください．

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

下記のコードを実行してGPU情報を確認します．



In [ ]:
!nvidia-smi

## 使用するデータセット

### データセット
今回の物体認識では，CIFAR-10データセットを使用します．CIFAR-10データセットは，飛行機や犬などの10クラスの物体が表示されている画像から構成されたデータセットです．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

### データセットのダウンロードと読み込み
実験に使用するCIFAR-10データセットを読み込みます．
１回の誤差を算出するデータ数 (ミニバッチサイズ) は，64とします．
まず，CIFAR-10データセットをダウンロードします．
次に，ダウンロードしたデータセットを読み込みます．
学習には，大量のデータを利用しますが，それでも十分ではありません． そこで，データ拡張 (data augmentation) により，データのバリエーションを増やします． 一般的な方法は，画像の左右反転，明るさ変換などです．

In [ ]:
batch_size = 64

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=20)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=20)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


### CIFAR-10データセットの表示
CIFAR-10データセットに含まれる画像を表示してみます．
ここでは，matplotlibを用いて複数の画像を表示させるプログラムを利用します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています．これは32x32ピクセルのカラー画像という意味になります．



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    npimg = ((npimg.transpose((1,2,0))  * [0.2023, 0.1994, 0.2010]) + [0.4914, 0.4822, 0.4465])  # unnormalize
    plt.imshow(npimg)
    plt.show()

# get some random training images
dataiter = iter(testloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## ネットワークモデルの定義
学習済みモデルにはResNet-18を利用して学習します．`pretrained = True`にすると，ImageNetで学習したモデルを利用できます．ここで，ImageNetは1,000クラスのデータセットです．すなわち，ImageNetで学習したResNet-18の出力層のユニット数は1,000になっています．ファインチューニングに利用するCIFAR-10データセットは10クラスなので，出力層のユニット数を変更します．

In [ ]:
model = models.resnet18(pretrained=True)
print("======== Original netowrk architecutre ========\n")
print(model)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
print("======== Fine-funing netowrk architecutre ========\n")
print(model)

model.to(device)

## 損失関数と最適化手法の定義
学習に使用する損失関数と最適化手法を定義します．
各更新において，学習用データと教師データをそれぞれ`inputs`と`targets`とします．
学習モデルに`inputs`を与えて，ResNetの出力を取得します．
ResNetの出力と教師ラベル`targets`との誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．
最適化手法には，確率的勾配降下法 (stochastic gradient descent: SGD) を用いて学習します．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．


In [ ]:
import torch.optim as optim
from torch.optim import SGD, lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1) # 50 < 75

# モデルの情報を表示
torchsummary.summary(model, (3, 32, 32))

## 学習
学習エポック数を10とします．
CIFAR-10データセットの学習データサイズを取得し，1エポック内における更新回数を求めます．
1エポック学習するごとに学習したモデルを評価し，最も精度の高いモデルが保存されます．

In [ ]:
epochs = 10
best_acc = 0  # best test accuracy

# Training
for epoch in range(epochs):
    #scheduler.step()
    train_running_loss = 0.0
    train_running_acc = 0.0

    # training
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    count = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # print statistics
        train_running_loss += loss
        train_running_acc += 100.*correct/total
        count += 1

    print('[Epoch %d] Train Loss: %.5f | Train Acc: %.3f%%'
                  % (epoch + 1, train_loss/count, train_running_acc/count))
    
    # testing
    model.eval() 
    with torch.no_grad():
        test_running_loss = 0.0
        test_running_acc = 0.0
        test_loss = 0
        correct = 0
        total = 0
        count = 0
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # print statistics
            test_running_loss += loss
            test_running_acc += 100.*correct/total
            count += 1

        print('Test Loss: %.5f | Test Acc: %.3f%%'
                      % (test_loss/count, test_running_acc/count))
        
    # save model
    if test_running_acc/count > best_acc:
        best_acc = max(test_running_acc/count, best_acc)
        PATH = './cifar_net.pth'
        torch.save(model.state_dict(), PATH)
    
print('Finished Training')

##テスト
学習したネットワークのテストデータに対する認識率の確認を行います．まず，学習したネットワークを評価するために保存したモデルをロードします．

In [ ]:
PATH = './cifar_net.pth'
model.load_state_dict(torch.load(PATH))

次に，学習したネットワークを用いて，テストデータに対する認識率の確認を行います．
`model.eval()`を適用することで，ネットワーク演算を評価モードへ変更します． これにより，学習時と評価時で挙動が異なる演算（dropout等）を変更することが可能です． また，`torch.no_grad()`を適用することで，学習時には必要になる勾配情報を保持することなく演算を行います．

In [ ]:
# testing
model.eval() 
with torch.no_grad():
    test_running_loss = 0.0
    test_running_acc = 0.0
    test_loss = 0
    correct = 0
    total = 0
    count = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # print statistics
        test_running_loss += loss
        test_running_acc += 100.*correct/total
        count += 1

    print('Test Loss: %.5f | Test Acc: %.3f%%'
                  % (test_loss/count, test_running_acc/count))

##Grad-CAMによるAttention mapの可視化
Grad-CAMを利用するために必要なツールをインストールします．
Grad-CAMは，`pytorch-gradcam`というツールをインストールすることで簡単に利用することができます．


In [ ]:
!pip install pytorch-gradcam

Grad-CAMによりAttention mapを可視化して，ネットワークの判断根拠を確認してみます． 再度，実行することで他のテストサンプルに対するAttention mapを可視化することができます． pred (prediction) は認識結果，conf (confidence) は認識結果に対する信頼度を示しています．

In [ ]:
from gradcam.utils import visualize_cam
from gradcam import GradCAM
import cv2
import numpy as np

transform_test = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=True, num_workers=20)

classes_list = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Grad-CAM
target_layer = model.layer4 # ex., layer1, layer2, layer3, layer3[1], layer4[0]
gradcam = GradCAM(model, target_layer)

softmax = nn.Softmax(dim=1)

def save_gradcam(gcam, raw_image):
    h, w, _ = raw_image.shape
    #gcam = cv2.resize(gcam, (w, h))
    gcam = gcam * 255.0
    gcam = np.uint8(gcam)
    gcam = gcam.transpose((1, 2, 0))
    #gcam = cv2.applyColorMap(gcam, cv2.COLORMAP_JET)
    #jet_map = cv2.addWeighted(raw_image, 0.4, gcam, 0.6, 0)
    v_list.append(raw_image)
    att_list.append(gcam)

for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    outputs = softmax(outputs)
    conf_data = outputs.data.topk(1, 1, True, True)
    _, predicted = outputs.max(1)
    d_inputs = inputs.data.cpu()
    d_inputs = d_inputs.numpy()
    in_b, in_c, in_y, in_x = inputs.shape

    v_list = []
    att_list = []
    for i in range(in_b):
        input = inputs[i,:,:,:]
        #input = torch.unsqueeze(input, dim=0)
        normed_torch_img = transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])(input)[None]

        v_img = d_inputs[i,:,:,:]
        v_img = v_img.transpose(1, 2, 0) * 255
        v_img = np.uint8(v_img)

        mask, _ = gradcam(normed_torch_img)
        heatmap, result = visualize_cam(mask, input)

        save_gradcam(result, v_img)
    break

# Show attention map
cols = 8
rows = 1

fig = plt.figure(figsize=(14, 3.0))
plt.title('Input image')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        cls = targets[c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        plt.title('{}'.format(classes_list[cls]))
        ax.imshow(v_list[cols * r + c])
        ax.set_axis_off()
plt.show()

fig = plt.figure(figsize=(14, 3.5))
plt.title('Attention map')
plt.axis("off")
for r in range(rows):
    for c in range(cols):
        pred = predicted[c].item()
        conf = conf_data[0][c].item()
        ax = fig.add_subplot(r+1, cols, c+1)
        ax.imshow(att_list[cols * r + c])
        plt.title('pred: {}\nconf: {:.2f}'.format(classes_list[pred], conf))
        ax.set_axis_off()
plt.show()

#課題
1. Attention mapを可視化する層を変更して，Attention mapの変化を確認してみましょう．


In [ ]:
#ここにコードを書く

# 参考文献
- [1] S. Ramprasaath, R., C. Michael, D. Abhishek,
V. Ramakrishna, P. Devi, and B.
Dhruv, "Grad-CAM: Visual explanations from deep networks
via gradient-based localization". In International Conference
on Computer Vision, pp. 618–626, 2017.

- [2] B. Zhou, A. Khosla, A. Lapedriza, A. Oliva,
and A. Torralba, "Learning deep features for discriminative
localization". In 2016 IEEE Conference on Computer
Vision and Pattern Recognition, pp. 2921–2929, 2016.

- [2] M. Lin, Q. Chen, and S. Yan, "Network in network".
In 2nd International Conference on Learning Representations,
Banff, AB, Canada, April 14-16, 2014, Conference
Track Proceedings, 2014.